## Data Berita CNN Indonesia

In [1]:
import builtwith

# Analisis teknologi yang digunakan
res = builtwith.parse('https://www.cnnindonesia.com/')
print(res)

{'cdn': ['CloudFlare'], 'databases': ['Firebase'], 'advertising-networks': ['Google AdSense'], 'tag-managers': ['Google Tag Manager'], 'javascript-frameworks': ['jQuery']}


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

#   Ambil data berita pada index website CnnIndonesia

In [3]:
BASE_URL = "https://www.cnnindonesia.com/"
headers = {"User-Agent": "Mozilla/5.0"}

def get_categories():
    response = requests.get(BASE_URL, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    categories = []
    for a in soup.select("a.navbar__item"):
        href = a.get("href")
        if href and href.startswith(BASE_URL):
            slug = href.replace(BASE_URL, "").strip("/")
            if slug and slug not in categories:
                categories.append(slug)
    return categories

def _extract_links_from_category_soup(soup, category):
    links = set()
    for a in soup.select("article a, .list-berita a, .headline a, .container__list__desc a"):
        href = a.get("href")
        if not href:
            continue
        if href.startswith("/"):
            href = BASE_URL + href.lstrip("/")
        # hanya link yang berada pada kategori yang sama
        if href.startswith(BASE_URL):
            relative = href.replace(BASE_URL, "")
            if relative.split("/")[0] == category:
                links.add(href)
    return list(links)

def _discover_index_base_for_category(category):
    # coba temukan link "Indeks" resmi untuk kategori tsb
    try:
        resp = requests.get(f"{BASE_URL}{category}", headers=headers, timeout=20)
        if resp.status_code != 200:
            return None
        soup = BeautifulSoup(resp.text, "html.parser")
        # cari tautan yang mengandung "/indeks/" dan masih dalam kategori yang sama
        for a in soup.find_all("a"):
            href = a.get("href")
            if not href:
                continue
            if href.startswith("/"):
                href = BASE_URL + href.lstrip("/")
            if not href.startswith(BASE_URL):
                continue
            rel = href.replace(BASE_URL, "")
            parts = rel.split("/")
            if len(parts) >= 3 and parts[0] == category and parts[1] == "indeks":
                # buang query jika ada, ambil base path saja
                base_no_query = href.split("?")[0].rstrip("/")
                return base_no_query
    except Exception:
        return None
    return None

def scrape_category(category, max_items=200):
    data = []
    seen_links = set()

    # temukan base indeks untuk kategori, misal:
    # https://www.cnnindonesia.com/internasional/indeks/6
    index_base = _discover_index_base_for_category(category)
    if not index_base:
        # fallback pola umum apabila tidak ditemukan
        index_base = f"{BASE_URL}{category}/indeks/6"

    page_number = 1
    while len(data) < max_items and page_number <= 100:
        url = f"{index_base}?page={page_number}"
        try:
            resp = requests.get(url, headers=headers, timeout=20)
            if resp.status_code != 200:
                break
            soup = BeautifulSoup(resp.text, "html.parser")
            links = _extract_links_from_category_soup(soup, category)
            if not links:
                # jika halaman tidak berisi link yang valid, hentikan
                break

            for link in links:
                if link in seen_links:
                    continue
                seen_links.add(link)

                try:
                    resp_detail = requests.get(link, headers=headers, timeout=20)
                    if resp_detail.status_code != 200:
                        continue
                    soup_detail = BeautifulSoup(resp_detail.text, "html.parser")

                    judul_tag = soup_detail.find("h1")
                    judul = judul_tag.get_text(strip=True) if judul_tag else ""

                    # kategori dari URL
                    kategori = None
                    try:
                        path = link.replace(BASE_URL, "")
                        kategori = path.split("/")[0] if path else None
                    except Exception:
                        kategori = None

                    paragraf = [p.get_text(strip=True) for p in soup_detail.select("div.detail-text p")]
                    isi = " ".join(paragraf)

                    if isi:
                        data.append({
                            "judul": judul,
                            "kategori": kategori,
                            "isi": isi,
                            "link": link
                        })

                    if len(data) >= max_items:
                        break

                    time.sleep(0.3)
                except Exception as e:
                    print(f"Gagal ambil {link}: {e}")

            if len(data) >= max_items:
                break

            page_number += 1
            time.sleep(0.7)
        except Exception:
            break

    return data

if __name__ == "__main__":
    all_data = []
    categories = get_categories()
    print("Kategori ditemukan:", categories)

    for cat in categories:
        print(f"Scraping kategori {cat} ...")
        items = scrape_category(cat, max_items=200)
        all_data.extend(items)
        print(f"  → {len(items)} berita diambil dari {cat}")
        time.sleep(1)

    # simpan ke DataFrame
    df = pd.DataFrame(all_data)
    pd.set_option("display.max_colwidth", 100)
    print(df.head(10))  # tampilkan 10 berita pertama

    df.to_csv("berita_cnn.csv", index=False, encoding="utf-8-sig")
    print("Selesai, data disimpan ke berita_cnn.csv")

Kategori ditemukan: ['nasional', 'internasional', 'ekonomi', 'olahraga', 'teknologi', 'otomotif', 'hiburan', 'gaya-hidup']
Scraping kategori nasional ...
  → 200 berita diambil dari nasional
Scraping kategori internasional ...
  → 200 berita diambil dari internasional
Scraping kategori ekonomi ...
  → 200 berita diambil dari ekonomi
Scraping kategori olahraga ...
  → 200 berita diambil dari olahraga
Scraping kategori teknologi ...
  → 200 berita diambil dari teknologi
Scraping kategori otomotif ...
  → 200 berita diambil dari otomotif
Scraping kategori hiburan ...
  → 200 berita diambil dari hiburan
Scraping kategori gaya-hidup ...
  → 200 berita diambil dari gaya-hidup
                                                                    judul  \
0       Dasco Beber Materi Rapat Bareng Menkeu Purbaya hingga Tito di DPR   
1  Nikita Mirzani Dituntut 11 Tahun Penjara & Denda Rp2 M Kasus Pemerasan   
2        Fadli Zon Pugar Situs Gunung Padang: Ini Piramida Asli Indonesia   
3            

In [4]:
df

,judul,kategori,isi,link
0,Dasco Beber Materi Rapat Bareng Menkeu Purbaya hingga Tito di DPR,nasional,Wakil Ketua DPRSufmi Dasco Ahmadmengungkap isi pertemuan tertutup dirinya dengan sejumlah menter...,https://www.cnnindonesia.com/nasional/20251009134232-32-1282746/dasco-beber-materi-rapat-bareng-...
1,Nikita Mirzani Dituntut 11 Tahun Penjara & Denda Rp2 M Kasus Pemerasan,nasional,Jaksa Penuntut Umum (JPU) menuntut majelis hakim Pengadilan Negeri (PN) Jakarta Selatan menghuku...,https://www.cnnindonesia.com/nasional/20251009130839-12-1282721/nikita-mirzani-dituntut-11-tahun...
2,Fadli Zon Pugar Situs Gunung Padang: Ini Piramida Asli Indonesia,nasional,Menteri KebudayaanFadli Zonmembeberkan rencana restrukturisasi Situs Megalitikum Gunung Padang d...,https://www.cnnindonesia.com/nasional/20251009141905-20-1282769/fadli-zon-pugar-situs-gunung-pad...
3,Tiga Purnawirawan TNI dan Polri Gabung Komite Otsus Papua,nasional,Sebanyak tiga purnawirawan TNI/Polri bergabung dalamKomite Eksekutif Percepatan Pembangunan Oton...,https://www.cnnindonesia.com/nasional/20251009125814-20-1282722/tiga-purnawirawan-tni-dan-polri-...
4,Ammar Zoni Diduga Jual Sabu hingga Ganja Sintetis di Rutan Salemba,nasional,AktorAmmar Zonikembali terlibat dalam kasus peredaran barang haramnarkotikasaat mendekam di Ruta...,https://www.cnnindonesia.com/nasional/20251009125445-12-1282717/ammar-zoni-diduga-jual-sabu-hing...
...,...,...,...,...
1595,Minat Pindah ke Selandia Baru? Sekarang Syaratnya Lebih Mudah,gaya-hidup,Selandia Barubaru saja mengumumkan bahwa negara ini melonggarkan persyaratan untuk mendapatkan i...,https://www.cnnindonesia.com/gaya-hidup/20250926205829-269-1278241/minat-pindah-ke-selandia-baru...
1596,"Cek, Begini Gejala Awal Munculnya Gagal Ginjal",gaya-hidup,"Gagal ginjaltermasuk penyakit serius yang dapat memburuk seiring waktu. Sayangnya, banyak orang ...",https://www.cnnindonesia.com/gaya-hidup/20250923154400-255-1276860/cek-begini-gejala-awal-muncul...
1597,Pertolongan Pertama saat Anak Alami Keracunan Makanan,gaya-hidup,Kasuskeracunanusai menyantap programmakan bergizi gratis(MBG) tengah menjadi sorotan publik bela...,https://www.cnnindonesia.com/gaya-hidup/20250926104108-255-1277922/pertolongan-pertama-saat-anak...
1598,Cerita Samantha 'Tuangkan' Kanker Payudara dalam Karya Seni,gaya-hidup,Berkesenian jadi sarana para pejuangkankerpayudara untuk menuangkan apa yang dirasakan. Hal ini ...,https://www.cnnindonesia.com/gaya-hidup/20250926142218-255-1278045/cerita-samantha-tuangkan-kank...


In [5]:

categories = get_categories()
print("Kategori ditemukan:", len(categories))
for i, cat in enumerate(categories, start=1):
    print(f"{i}. {cat}")

Kategori ditemukan: 8
1. nasional
2. internasional
3. ekonomi
4. olahraga
5. teknologi
6. otomotif
7. hiburan
8. gaya-hidup
